Import Requirements

In [1]:
import pyodbc
import re

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

Connection

In [3]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [4]:
tables = engine.table_names()

C:\Users\eats\AppData\Local\Temp\ipykernel_10156\863585613.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


Transform Function

In [5]:
def pick_date(str_date: str):
    return str_date[:10]

# menghitung berapa lama pengalaman berdasarkan tanggal mulai dan selesai
def get_experience_time(col1: pd.Series, col2: pd.Series):
    col1.map(pd.to_datetime)
    col2.map(pd.to_datetime)
    
    print(col1)
    print(col2)

    return (col1-col2).dt.months

# membenarkan tahun yang tidak jelas di database, eg: 212 --> 2012 dan 15 --> 2015
def repair_years(str_date: str):
    print(str_date)
    str_date = str_date.split('-')
    
    year = str(int(str_date[0]))
    if len(year) == 3:
        year = year[0] + '0' + year[1:]
        print(year)
    elif len(year) == 2:
        year = '20' + year
        print(year)
    else:
        pass
        
    str_date[0] = year
    return '-'.join(str_date)

def years_to_zero(str_date: str):
    str_date = str_date.split('-')

    year = int(str_date[0])
    if year < 1990:
        return '0'
    elif year > 2023:
        return '0'
    
    return '-'.join(str_date)

def html_cleaner(text: str):
    re_html = re.compile(r'<[^>]+>')
    return re_html.sub('', text)

def remove_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

# belum dipake
def clearing_nationality(text: str):
    text = text.strip().lower()

    text_dict = ('wni', 'indonesia', 'jawa', 'java')

    return 'indonesia' if text in text_dict else text

def get_age(dob):
    return (datetime.now() - dob) / np.timedelta64(1, 'Y')

def remove_badyears(str_date, min_border, max_border):
    year = str_date.split('-')[0]

    if max_border < int(year) or int(year) < min_border:
        return np.NaN
    return str_date

def show_unique(df, batas=15):
    for col in df.columns:
        uniq = df[col].unique()
        if len(uniq) <= batas:
            print(col, '>', uniq)

### Tables

In [6]:
df_applicant = pd.DataFrame(engine.execute('SELECT ApplicantID, DiseaseHistory, Dob, ExpectedSalary, IsAbleToWorkRemote, CityID, ProvinceID, DriverLicenseType, Gender, IsUsingGlasses, Height, MaritalStatus, Nationality, Strengthness, Weaknesses, TypeOfVehicle FROM Applicant'))
df_applicant_education = pd.DataFrame(engine.execute('SELECT ApplicantEducationID, ApplicantID, DateStart, DateEnd, EducationInstituteName, Score, EducationLevelID, MajorID, Degree FROM ApplicantEducation'))
df_applicant_experience = pd.DataFrame(engine.execute('SELECT ApplicantExperienceID, ApplicantID, DateFrom, DateTo, Industry, CompanyName, JobDescription, Position, Salary FROM ApplicantExperience'))
df_applicant_document = pd.DataFrame(engine.execute('SELECT ApplicantID, DocumentName FROM ApplicantDocument'))
df_applicant_certificate = pd.DataFrame(engine.execute('SELECT ApplicantID, Description FROM ApplicantCertificate'))

df_pipeline = pd.DataFrame(engine.execute('SELECT PipelineID, ApplicantID, JobID, StageID FROM Pipeline'))
df_stage = pd.DataFrame(engine.execute('SELECT StageID, Label FROM Stage'))

df_job = pd.DataFrame(engine.execute('SELECT * FROM Job'))
df_function_position = pd.DataFrame(engine.execute('SELECT FunctionPositionID, FunctionPositionName FROM FunctionPosition'))
df_department = pd.DataFrame(engine.execute('SELECT DepartmentID, Name FROM Department'))
df_city = pd.DataFrame(engine.execute('SELECT CityID, Name AS CityName FROM City'))
df_province = pd.DataFrame(engine.execute('SELECT ProvinceID, Name AS ProvinceName FROM Province'))
df_major = pd.DataFrame(engine.execute('SELECT MajorID, MajorName FROM Major'))
df_education_level = pd.DataFrame(engine.execute('SELECT EducationLevelID, EducationLevelName FROM EducationLevel'))
df_company = pd.DataFrame(engine.execute('SELECT CompanyID, Name AS CompanyName FROM Company'))

In [7]:
df_job = df_job[['JobID', 'Description', 'EducationLevelID', 'FunctionPositionID', 'JobStatus', 'JobTitle', 'Requirement', 'SalaryMax', 'SalaryMin', 'DepartmentID', 'CityID', 'CompanyID', 'ProvinceID', 'MajorID', 'UsiaMax', 'UsingGlasses', 'IQMin', 'DriverLicenseType', 'PostedDate', 'PublishDate', 'ClosedDate', 'JobStatus']]

In [8]:
df_job[df_job.JobID.duplicated()]

,JobID,Description,EducationLevelID,FunctionPositionID,JobStatus,JobTitle,Requirement,SalaryMax,SalaryMin,DepartmentID,CityID,CompanyID,ProvinceID,MajorID,UsiaMax,UsingGlasses,IQMin,DriverLicenseType,PostedDate,PublishDate,ClosedDate,JobStatus


In [9]:
df_job.isna().sum()

JobID                    0
Description              2
EducationLevelID         0
FunctionPositionID       0
JobStatus                0
JobTitle                 0
Requirement              2
SalaryMax                0
SalaryMin                0
DepartmentID             0
CityID                   0
CompanyID                0
ProvinceID               0
MajorID                  0
UsiaMax                 16
UsingGlasses             4
IQMin                    0
DriverLicenseType     1007
PostedDate               0
PublishDate              0
ClosedDate               0
JobStatus                0
dtype: int64

In [10]:
df_job.Description.fillna('', inplace=True)
df_job.Requirement.fillna('', inplace=True)
df_job.UsiaMax.fillna(65, inplace=True)
df_job.UsingGlasses.fillna(False, inplace=True)
df_job.DriverLicenseType.fillna('', inplace=True)

Check

In [12]:
df_education_level

,EducationLevelID,EducationLevelName
0,4,D3
1,3,D4
2,9,None
3,2,S1
4,1,S2
5,10,S3
6,8,SD
7,5,SMA
8,6,SMK
9,7,SMP


In [13]:
df_merged = pd.merge(df_job, df_education_level, on=['EducationLevelID'])
df_merged = pd.merge(df_merged, df_city, on=['CityID'])
df_merged = pd.merge(df_merged, df_province, on=['ProvinceID'])
df_merged = pd.merge(df_merged, df_company, on=['CompanyID'])
df_merged = pd.merge(df_merged, df_function_position, on=['FunctionPositionID'])
df_merged = pd.merge(df_merged, df_department, on=['DepartmentID'])
df_merged = pd.merge(df_merged, df_major, on=['MajorID'])

df_merged.drop(columns=['EducationLevelID', 'CityID', 'ProvinceID', 'CompanyID', 'FunctionPositionID', 'DepartmentID', 'MajorID'], inplace=True)

Check

In [14]:
df_merged.head(3)

,JobID,Description,JobStatus,JobTitle,Requirement,SalaryMax,SalaryMin,UsiaMax,UsingGlasses,IQMin,DriverLicenseType,PostedDate,PublishDate,ClosedDate,JobStatus,EducationLevelName,CityName,ProvinceName,CompanyName,FunctionPositionName,Name,MajorName
0,1,Under Asst. Front Office Manager at Gunawangsa...,Close,NIGHT AUDIT,"<p>\r\n\r\n</p><div style=""language:en-US;marg...",3600000.0,3300000.0,0.0,False,0,,2018-04-10 00:00:00.0000000,0001-01-01 00:00:00.0000000,2018-05-19 08:54:53.3799529,Close,D3,SURABAYA,JAWA TIMUR,GUNAWANGSA HOTEL MERR,NIGHT AUDIT,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN
1,1490,<p>Test</p>,Close,Test,<p>Test</p>,0.0,0.0,0.0,False,0,,2020-07-14 00:00:00.0000000,0001-01-01 00:00:00.0000000,2020-09-27 10:46:45.7228097,Close,S2,SURABAYA,JAWA TIMUR,GUNAWANGSA HOTEL MERR,NIGHT AUDIT,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN
2,48,<p>Becoming Executive Housekeeping For Gunawan...,Close,EXECUTIVE HOUSEKEEPING,<p>Male<br>Experience in hospitality industry ...,6000000.0,3587000.0,0.0,False,0,,2018-07-03 00:00:00.0000000,0001-01-01 00:00:00.0000000,2018-09-12 11:35:14.7251826,Close,D3,SURABAYA,JAWA TIMUR,GUNAWANGSA HOTEL MERR,EXECUTIVE HOUSEKEEPING,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN


In [27]:
df_merged.drop(
    index=df_merged.index[df_merged.JobTitle.map(str.lower).str.contains('test|123')].values, inplace=True
)

In [19]:
df_bad_salary = df_merged[(df_merged.SalaryMin < 1_000_000) & (df_merged.SalaryMin != 0) & (df_merged.SalaryMax != 0)]

In [31]:
df_bad_description = df_merged[(df_merged.Description == '') | (df_merged.Requirement == '')]

In [42]:
df_bad_salary.to_csv('data/df_bad_salary.csv', index=False)
df_bad_description.to_csv('data/df_bad_description.csv', index=False)

In [46]:
df_merged[(df_merged.SalaryMax < 500) & (df_merged.SalaryMax != 0)]

,JobID,Description,JobStatus,JobTitle,Requirement,SalaryMax,SalaryMin,UsiaMax,UsingGlasses,IQMin,DriverLicenseType,PostedDate,PublishDate,ClosedDate,JobStatus,EducationLevelName,CityName,ProvinceName,CompanyName,FunctionPositionName,Name,MajorName
356,3078,<p>- Mempersiapkan mobil yang akan digunakan u...,Close,DRIVER,<p>- Memiliki sim A / B1<br>- Pria usia 25 - 3...,2.0,1.0,0.0,False,0,,2023-02-22 00:00:00.0000000,2023-02-22 00:00:00.0000000,2023-02-24 10:15:54.8651553,Close,None,SURABAYA,JAWA TIMUR,WARNA WARNI MEDIA SURABAYA,DRIVER,OPERASIONAL WARNA WARNI MEDIA SURABAYA,SEMUA JURUSAN
372,3085,"<div data-contents=""true""><div class="""" data-b...",Publish,SURVEYOR LAPANGAN,"<div data-contents=""true""><ul><li><span data-o...",1.0,0.0,0.0,True,0,,2023-03-13 00:00:00.0000000,2023-03-13 00:00:00.0000000,0001-01-01 00:00:00.0000000,Publish,None,SURABAYA,JAWA TIMUR,WARNA WARNI MEDIA SURABAYA,LOKASI,PRODUKSI WARNA WARNI MEDIA SURABAYA,SEMUA JURUSAN
503,3077,"<ul class=""public-DraftStyleDefault-ul"" data-o...",Not Publish,SENIOR ACCOUNTING,"<div data-contents=""true""><ul><li><span data-o...",2.0,1.0,35.0,True,0,,2023-02-21 00:00:00.0000000,2023-02-22 00:00:00.0000000,0001-01-01 00:00:00.0000000,Not Publish,None,SURABAYA,JAWA TIMUR,WARNA WARNI MEDIA SURABAYA,FINANCE & ACCOUNTING,MARKETING PT CENTRAL INVESTINDO,AKUNTANSI
522,3076,"<ul class=""public-DraftStyleDefault-ul"" data-o...",Publish,SENIOR ACCOUNTING,"<div data-contents=""true""><ul><li><span data-o...",2.0,1.0,35.0,True,0,,2023-02-21 00:00:00.0000000,2023-02-21 00:00:00.0000000,0001-01-01 00:00:00.0000000,Publish,None,SURABAYA,JAWA TIMUR,WARNA WARNI MEDIA SURABAYA,FINANCE & ACCOUNTING,ACCOUNTING WARNA WARNI MEDIA SURABAYA,AKUNTANSI
872,3082,<ul><li>Mengerjakan ETL</li><li>Menyelesaikan ...,Close,IT ARTIFICIAL INTELLIGENCE,<ul><li>S1 Informatika/komputer/bidang lain ya...,2.0,1.0,0.0,True,0,,2023-03-02 00:00:00.0000000,2023-03-02 00:00:00.0000000,2023-03-02 11:57:26.9022685,Close,None,SURABAYA,JAWA TIMUR,WARNA WARNI MEDIA SURABAYA,IT,IT WARNA WARNI MEDIA SURABAYA,TEKNIK INFORMATIKA
921,3079,"<div data-contents=""true""><ul class=""public-Dr...",Close,PENGAWAS LAPANGAN,"<div data-contents=""true""><div class="""" data-b...",2.0,1.0,35.0,False,0,,2023-02-24 00:00:00.0000000,2023-02-24 00:00:00.0000000,2023-03-02 15:13:22.3099917,Close,S1,SURABAYA,JAWA TIMUR,WARNA WARNI MEDIA SURABAYA,LOKASI,PRODUKSI WARNA WARNI MEDIA SURABAYA,TEKNIK SIPIL
